<a href="https://colab.research.google.com/github/HitaishiD/Lane_segmentation_project/blob/main/LLM_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Tokenization

In [ ]:
!pip install transformers --upgrade

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig

In [ ]:
# Load model and tokenizer
model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3-mini-4k-instruct",
    device_map="cuda",
    torch_dtype="auto",
    trust_remote_code=True,
)
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-4k-instruct")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/967 [00:00<?, ?B/s]

configuration_phi3.py:   0%|          | 0.00/11.2k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3-mini-4k-instruct:
- configuration_phi3.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_phi3.py:   0%|          | 0.00/73.2k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/3.44k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.94M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/306 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/599 [00:00<?, ?B/s]

In [ ]:
from transformers import pipeline

# Create a pipeline
generation_config = GenerationConfig(
    max_new_tokens=500,
    do_sample=False,
    use_cache=False
)
generator = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    generation_config=generation_config,
    return_full_text=False
)

Device set to use cuda


In [ ]:
# Prompt (user input/query)
messages = [
    {"role":"user", "content": "Create a funny joke about chickens."}
]

# Generate output
output = generator(messages)
print(output[0]['generated_text'])

 Why did the chicken join the band? Because it had the drumsticks!


In [ ]:
prompt = "Write an email to explain the tragic gardening mishap."

# Tokenize the input prompt
input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to("cuda")

# Generate the text
generation_output = model.generate(
    input_ids=input_ids,
    max_new_tokens=50,
    use_cache = False
)

# Print the output
print(tokenizer.decode(generation_output[0]))

Write an email to explain the tragic gardening mishap.

Subject: Apology for the Gardening Mishap

Dear [Recipient's Name],

I hope this email finds you well. I am writing to express my deepest apologies for the unfortunate


In [ ]:
input_ids

tensor([[14350,   385,  4876,   304,  5649,   278, 25305,   293, 16423,   292,
           286,   728,   481, 29889]], device='cuda:0')

In [ ]:
tokenizer.decode(16423)

'garden'

In [ ]:
colors_list = [
'102;194;165', '252;141;98', '141;160;203',
'231;138;195', '166;216;84', '255;217;47'
]

def show_tokens(sentence, tokenizer_name):
  tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)
  token_ids = tokenizer(sentence).input_ids
  for idx, t in enumerate(token_ids):
    print(
    f'\x1b[0;30;48;2;{colors_list[idx % len(colors_list)]}m' +
    tokenizer.decode(t) +
    '\x1b[0m',
    end=' '
    )

In [ ]:
show_tokens("Tell me something about CARS", "microsoft/Phi-3-mini-4k-instruct")

Tell me something about C AR S 

### Embedding

In [ ]:
!pip install --upgrade numpy
!pip install --force-reinstall --no-cache-dir gensim

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 83.6 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gensim 4.3.3 requires numpy<2.0,>=1.18.5, but you have numpy 2.2.5 which is incompatible.
numba 0.60.0 requires numpy<2.1,>=1.22, but you have numpy 2.2.5 which is incompatible.
tensorflow 2.18.0 requires numpy<2.1.0,>=1.26.0, but you have numpy 2.2.5 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 139.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 126.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 308.9 MB/s eta 0:00:0

In [ ]:
import gensim.downloader as api

In [ ]:
# Download embeddings (66MB, glove, trained on wikipedia, vector size: 50)
# Other options include "word2vec-google-news-300"
# More options at https://github.com/RaRe-Technologies/gensim-data
model = api.load("glove-wiki-gigaword-50")

[==================================================] 100.0% 66.0/66.0MB downloaded


In [ ]:
model.most_similar([model['canada']], topn=11)

[('canada', 1.0000001192092896),
 ('united', 0.8320111036300659),
 ('canadian', 0.8224480748176575),
 ('states', 0.8098676204681396),
 ('australia', 0.7994687557220459),
 ('zealand', 0.7906773090362549),
 ('britain', 0.7580573558807373),
 ('pacific', 0.7389345765113831),
 ('atlantic', 0.7299982309341431),
 ('european', 0.7249375581741333),
 ('europe', 0.7197747230529785)]

### Simple RAG with Cohere API

In [3]:
!pip install cohere PyPDF2==3.0.1 faiss-cpu

  Using cached cohere-5.15.0-py3-none-any.whl.metadata (3.4 kB)
  Using cached pypdf2-3.0.1-py3-none-any.whl.metadata (6.8 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.5/259.5 kB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 64.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 88.6 MB/s eta 0:00:00


In [4]:
import cohere
import PyPDF2
import faiss
import numpy as np

In [5]:
# --- SETUP ---
COHERE_API_KEY = '0jACV23n5ZKqpk7F8aZPpje7kVUM16PIUvLBuc4p'
co = cohere.Client(COHERE_API_KEY)

In [6]:
# --- STEP 1: Read PDF ---
def extract_text_from_pdf(pdf_path):
    reader = PyPDF2.PdfReader(pdf_path)
    text = ""
    for page in reader.pages:
        text += page.extract_text() + "\n"
    return text

In [7]:
# --- STEP 2: Split into chunks ---
def split_into_chunks(text, max_chars=500):
    return [text[i:i+max_chars] for i in range(0, len(text), max_chars)]

In [14]:
# --- STEP 3: Get embeddings from Cohere ---
def get_embeddings(text_chunks):
    response = co.embed(texts=text_chunks,
                        model="embed-english-v3.0",
                        input_type="search_document")
    return np.array(response.embeddings)

In [9]:
# --- STEP 4: Build FAISS index ---
def build_faiss_index(embeddings):
    dim = embeddings.shape[1]
    index = faiss.IndexFlatL2(dim)
    index.add(embeddings)
    return index

In [18]:
# --- STEP 5: Search similar chunks ---
def search_similar_chunks(query, text_chunks, embeddings, index, k=3):
    query_embedding = co.embed(texts=[query],
                               model="embed-english-v3.0",
                               input_type="search_query").embeddings[0]
    D, I = index.search(np.array([query_embedding]), k)
    return [text_chunks[i] for i in I[0]]

In [22]:
def generate_answer(query, context):
    prompt = f"Context: {context}\n\nQuestion: {query}\nAnswer:"
    response = co.chat(
        model="command-nightly",  # "command-nightly" if you're on the free tier
        message=query,
        documents=[{'text':context}],
        temperature=0.5,
    )
    return response.text.strip()

In [23]:
# --- MAIN FUNCTION ---
def run_rag(pdf_path, query):
    text = extract_text_from_pdf(pdf_path)
    chunks = split_into_chunks(text)
    embeddings = get_embeddings(chunks)
    index = build_faiss_index(embeddings)
    context = " ".join(search_similar_chunks(query, chunks, embeddings, index))
    answer = generate_answer(query, context)
    return answer

In [24]:
#--- USAGE EXAMPLE ---
answer = run_rag("F1test.pdf", "What is the main idea of this document?")
print(answer)

The main idea of this document is to provide a brief overview of the history and significance of Formula One (F1) racing. It highlights the origins of the sport, its evolution from earlier championships, and its establishment as the highest class of worldwide racing for open-wheel single-seater formula racing cars. The document also mentions the Fédération Internationale de l'Automobile (FIA), which sanctions F1 races, and the FIA Formula One World Championship, which has been a premier form of motorsport since its inaugural running in 1950.


### LangChain-based RAG pipeline

In [1]:
pip install langchain langchain-community pypdf chromadb sentence-transformers huggingface_hub accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 35.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.3/302.3 kB 22.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.9/18.9 MB 94.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 21.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 78.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 94.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.4/188.4 kB 14.7 MB/s eta 0:

In [11]:
!pip install rank_bm25

In [1]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma
from langchain.retrievers import BM25Retriever, EnsembleRetriever
from langchain.llms import HuggingFacePipeline
from langchain.chains import RetrievalQA
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import os

#### Define all functions

In [2]:
# --- STEP 1: Load multiple PDFs ---
def load_pdfs_from_folder(folder_path):
    all_docs = []
    for filename in os.listdir(folder_path):
        if filename.endswith(".pdf"):
            loader = PyPDFLoader(os.path.join(folder_path, filename))
            all_docs.extend(loader.load())
    return all_docs

In [15]:
all_docs = load_pdfs_from_folder("/content/sample_data/testfiles")
print(all_docs[0])

page_content='Manufacturers' return 
 
Michael Schumacher (pictured here in 2001) won five consecutive titles with Ferrari. 
Michael Schumacher and Ferrari won five consecutive Drivers' Championships (2000–2004) 
and six consecutive Constructors' Championships (1999–2004). Schumacher set many new 
records, including those for Grand Prix wins (91, since beaten by Lewis Hamilton), wins in a 
season (13, since beaten by Max Verstappen), and most Drivers' Championships (seven, tied 
with Lewis Hamilton as of 2021).[37] Schumacher's championship streak ended on 25 
September 2005, when Renault driver Fernando Alonso became Formula One's youngest 
champion at that time (until Lewis Hamilton in 2008 and followed by Sebastian Vettel in 2010). 
During 2006, Renault and Alonso won both titles again. Schumacher retired at the end of 2006 
after 16 years in Formula One, but came out of retirement for the 2010 season, racing for the 
newly formed Mercedes works team, following the rebrand of Brawn 

In [17]:
print(len(all_docs))

10


In [3]:
# --- STEP 2: Split text into smart chunks ---
def split_documents(documents):
    splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
    return splitter.split_documents(documents)

In [16]:
splitted_doc = split_documents(all_docs)
print(splitted_doc[0])

page_content='Manufacturers' return 
 
Michael Schumacher (pictured here in 2001) won five consecutive titles with Ferrari. 
Michael Schumacher and Ferrari won five consecutive Drivers' Championships (2000–2004) 
and six consecutive Constructors' Championships (1999–2004). Schumacher set many new 
records, including those for Grand Prix wins (91, since beaten by Lewis Hamilton), wins in a 
season (13, since beaten by Max Verstappen), and most Drivers' Championships (seven, tied' metadata={'producer': 'Microsoft® Word for Microsoft 365', 'creator': 'Microsoft® Word for Microsoft 365', 'creationdate': '2025-05-06T16:23:23-04:00', 'author': 'Prisha Dhoowooah', 'moddate': '2025-05-06T16:23:23-04:00', 'source': '/content/sample_data/testfiles/F1test3.pdf', 'total_pages': 3, 'page': 0, 'page_label': '1'}


In [18]:
print(len(splitted_doc))

72


In [4]:
# --- STEP 3: Generate embeddings using MiniLM ---
def get_hf_embeddings():
    return HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

In [5]:
# --- STEP 4: Create vector store (ChromaDB) ---
def create_vectorstore(chunks, embeddings, persist_dir="chroma_db"):
    vectordb = Chroma.from_documents(chunks, embeddings, persist_directory=persist_dir)
    vectordb.persist()
    return vectordb

In [24]:
embeddings = get_hf_embeddings()
vectordb = create_vectorstore(splitted_doc, embeddings)

data = vectordb.get(include=["documents", "metadatas", "embeddings"])

print("📄 Sample Document:", data["documents"][0])
print("🧠 Embedding Vector (first 5 dims):", data["embeddings"][0][:5])
print("🔖 Metadata:", data["metadatas"][0])
print("🆔 ID:", data["ids"][0]) # IDs are always present in the return dictionary

📄 Sample Document: Manufacturers' return 
 
Michael Schumacher (pictured here in 2001) won five consecutive titles with Ferrari. 
Michael Schumacher and Ferrari won five consecutive Drivers' Championships (2000–2004) 
and six consecutive Constructors' Championships (1999–2004). Schumacher set many new 
records, including those for Grand Prix wins (91, since beaten by Lewis Hamilton), wins in a 
season (13, since beaten by Max Verstappen), and most Drivers' Championships (seven, tied
🧠 Embedding Vector (first 5 dims): [-0.02149874  0.01316393 -0.03308622 -0.01627643 -0.05734529]
🔖 Metadata: {'creator': 'Microsoft® Word for Microsoft 365', 'total_pages': 3, 'author': 'Prisha Dhoowooah', 'source': '/content/sample_data/testfiles/F1test3.pdf', 'page': 0, 'moddate': '2025-05-06T16:23:23-04:00', 'page_label': '1', 'creationdate': '2025-05-06T16:23:23-04:00', 'producer': 'Microsoft® Word for Microsoft 365'}
🆔 ID: fb3c0902-2435-461e-a050-470f7ed24dab


In [6]:
# --- STEP 5: Setup Hybrid Retriever ---
def create_hybrid_retriever(chunks, vectordb):
    bm25 = BM25Retriever.from_documents(chunks)
    bm25.k = 3
    vectordb_retriever = vectordb.as_retriever(search_kwargs={"k": 3})

    ensemble = EnsembleRetriever(retrievers=[bm25, vectordb_retriever], weights=[0.5, 0.5])
    return ensemble

In [27]:
ens_retriever = create_hybrid_retriever(splitted_doc, vectordb)
print(ens_retriever)

retrievers=[BM25Retriever(vectorizer=<rank_bm25.BM25Okapi object at 0x7bd58f076850>, k=3), VectorStoreRetriever(tags=['Chroma', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x7bd58f315e50>, search_kwargs={'k': 3})] weights=[0.5, 0.5]


In [7]:
# --- STEP 6: Load free local LLM from HuggingFace ---
def load_local_llm(model_id="TinyLlama/TinyLlama-1.1B-Chat-v1.0", max_new_tokens=256):
    tokenizer = AutoTokenizer.from_pretrained(model_id)
    model = AutoModelForCausalLM.from_pretrained(
        model_id,
        device_map="cpu",
        trust_remote_code=True,
        torch_dtype="auto"
    )

    pipe = pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        max_new_tokens=max_new_tokens,
        temperature=0.5,
        do_sample=True
    )

    return HuggingFacePipeline(pipeline=pipe)

In [8]:
# --- STEP 7: Build the RAG Chain ---
def build_qa_chain(llm, retriever):
    return RetrievalQA.from_chain_type(llm=llm, retriever=retriever, chain_type="stuff")

#### Run full RAG pipeline

In [12]:
# --- MAIN: Run full pipeline ---
def run_rag_pipeline(pdf_folder, query):
    # Load and chunk
    docs = load_pdfs_from_folder(pdf_folder)
    chunks = split_documents(docs)

    # Embeddings and vector store
    embeddings = get_hf_embeddings()
    vectordb = create_vectorstore(chunks, embeddings)

    # Hybrid retriever
    retriever = create_hybrid_retriever(chunks, vectordb)

    # Load LLM
    llm = load_local_llm()

    # QA Chain
    qa_chain = build_qa_chain(llm, retriever)

    # Query the system
    answer = qa_chain.run(query)
    return answer

#### Example 1: correct answer

In [13]:
# --- EXAMPLE USAGE ---
answer = run_rag_pipeline("/content/sample_data/testfiles", "What is the key point across these PDFs?")
print(answer)

tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

Device set to use cpu
<ipython-input-7-1c8c79f62ecb>:20: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  return HuggingFacePipeline(pipeline=pipe)
<ipython-input-12-16e36ab541b7>:21: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  answer = qa_chain.run(query)


Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

The formation of the Fédération Internationale du Sport Automobile (FISA) in 1979 set off the 
FISA–FOCA war, during which FISA and its president Jean-Marie Balestre argued repeatedly 
with FOCA over television revenues and technical regulations.[26] The Guardian said that 
Ecclestone and Max Mosley "used [FOCA] to wage a guerrilla war with a very long-term aim in 
view". FOCA threatened to establish a rival series and boycotted a Grand Prix, and FISA

Despite the entry of these new teams, the proposed cost-cap was repealed and these teams—
which did not have the budgets of the midfield and top-order teams—ran around at the back of 
the field until they collapsed; HRT in 2012, Caterham (formerly Lotus) in 2014 and Manor 
(formerly Virgin, then Marussia), having survived falling into administration in 2014, at the end of 
201

#### Example 2: correct answer but extra query

In [28]:
# --- EXAMPLE USAGE ---
answer = run_rag_pipeline("/content/sample_data/testfiles", "Who is Charles Leclerc?")
print(answer)

Device set to use cpu


Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

Big business 
Beginning in the 1970s, Bernie Ecclestone rearranged the management of Formula One's 
commercial rights; he is widely credited with transforming the sport into the multi-billion dollar 
business it now is.[23][24] When Ecclestone bought the Brabham team during 1971, he gained 
a seat on the Formula One Constructors' Association, and in 1978, he became its president.[25] 
Previously, the circuit owners controlled the income of the teams and negotiated with each

Formula One (F1) is the highest class of worldwide racing for open-wheel single-seater formula 
racing cars sanctioned by the Fédération Internationale de l'Automobile (FIA). The FIA Formula 
One World Championship has been one of the world's premier forms of motorsport since its 
inaugural running in 1950 and is often considered to be the pinnacle of mo

#### Example 3: incorrect answer

In [29]:
# --- EXAMPLE USAGE ---
answer = run_rag_pipeline("/content/sample_data/testfiles", "Which team will enter the Formula 1 competition in 2026?")
print(answer)

Device set to use cpu


Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

spare.[69][70][71][72] 
 
In 2023 the FIA opened applications for new teams to enter Formula 1 in the then near 
future.[73] Of the teams that applied, only Andretti were approved by the FIA, with them then 
being rejected by Formula One Management, though they have launched an appeal.[74][75] 
 
In early 2024, the Formula One landscape underwent a significant change in the sphere of team 
sponsorships and collaborations. Having competed for five seasons under the Alfa Romeo

More stringent safety requirements 
Tighter budget caps 
In November 2024, General Motors reached an agreement to enter Formula 1 in 2026 with its 
Cadillac brand.[82]

sponsorships and collaborations. Having competed for five seasons under the Alfa Romeo 
name, Sauber introduced a title partnership with the online casino Stake.com, resulting in the 
te

#### Try map reduce method

In [45]:
from langchain.chains import MapReduceDocumentsChain, ReduceDocumentsChain
from langchain.chains.llm import LLMChain
from langchain.prompts import PromptTemplate

In [52]:
from langchain.chains import MapReduceDocumentsChain, ReduceDocumentsChain, StuffDocumentsChain
from langchain.chains.llm import LLMChain
from langchain.prompts import PromptTemplate

def build_qa_chain2(llm, retriever):
    # MAP prompt (per document chunk)
    map_prompt = PromptTemplate.from_template("""
    Use the following context to answer the question.
    If the context does not contain the answer, respond with "I don't know".

    Context:
    {context}

    Question:
    {question}

    Answer:
    """)
    map_chain = LLMChain(llm=llm, prompt=map_prompt)

    # REDUCE prompt (combine all partial answers)
    reduce_prompt = PromptTemplate.from_template("""
    The following are answers from multiple sources. Please summarize them into a single final answer.

    Answers:
    {context}

    Final Answer:
    """)
    reduce_chain = LLMChain(llm=llm, prompt=reduce_prompt)

    # Initialize StuffDocumentsChain with reduce_chain to create combine_documents_chain
    combine_documents_chain = StuffDocumentsChain(
        llm_chain=reduce_chain,
        document_variable_name="context"
    )

    # Initialize ReduceDocumentsChain directly within MapReduceDocumentsChain
    map_reduce_chain = MapReduceDocumentsChain(
        llm_chain=map_chain,
        # Use combine_documents_chain instead of individual attributes
        reduce_documents_chain=combine_documents_chain,
        document_variable_name="context"
    )

    return RetrievalQA(retriever=retriever, combine_documents_chain=map_reduce_chain)

In [53]:
# --- MAIN: Run full pipeline ---
def run_rag_pipeline(pdf_folder, query):
    # Load and chunk
    docs = load_pdfs_from_folder(pdf_folder)
    chunks = split_documents(docs)

    # Embeddings and vector store
    embeddings = get_hf_embeddings()
    vectordb = create_vectorstore(chunks, embeddings)

    # Hybrid retriever
    retriever = create_hybrid_retriever(chunks, vectordb)

    # Load LLM
    llm = load_local_llm()

    # QA Chain
    qa_chain = build_qa_chain2(llm, retriever)

    # Query the system
    answer = qa_chain.run(query)
    return answer